In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# 加载数据集
DataSet = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = DataSet.load_data()

In [3]:
# 数据预处理
train_images, test_images = train_images / 255.0, test_images / 255.0

In [4]:
class SparseConv2D(layers.Layer):
    def __init__(self, filters, kernel_size, p, **kwargs):
        super(SparseConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.p = tf.Variable(float(p), trainable=False)  # 将 p 转换为浮点数类型
        self.counter = tf.Variable(0, trainable=False, dtype=tf.int32)  # 初始化计数器

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.filters,),
                                    initializer='zeros',
                                    trainable=True)

    @tf.function
    def call(self, inputs, training=None):
        if training:
            mask = tf.random.uniform(shape=(self.filters,), minval=0, maxval=1)
            mask = tf.cast(mask < self.p, dtype=tf.float32)
            mask = tf.reshape(mask, [1, 1, 1, self.filters])
            #self.counter.assign_add(1)  # 更新计数器
            #tf.print("\nP is", self.p)  # 使用 tf.print
        else:
            mask = tf.ones([1, 1, 1, self.filters], dtype=tf.float32) * self.p
    
        sparse_kernel = self.kernel * mask
        conv = tf.nn.conv2d(inputs, sparse_kernel, strides=[1, 1, 1, 1], padding='SAME')
        return tf.nn.bias_add(conv, self.bias)

    @tf.function
    def update_p(self, new_p):
        self.p.assign(float(new_p))  # 使用 assign 更新 tf.Variable 的值，并转换为浮点数
        #tf.print("\nEpoch counter is", self.counter)  # 使用 tf.print
        #tf.print("\nP is", self.p)  # 使用 tf.print

In [5]:
# 创建模型使用稀疏卷积层
inputs = tf.keras.Input(shape=(32, 32, 3))  # 输入层，形状为 (32, 32, 3)
x = SparseConv2D(filters=32, kernel_size=3, p=1, name='sparse_conv2d_1')(inputs)  # 第一个稀疏卷积层
x = layers.Activation('relu')(x)  # 激活函数
x = layers.MaxPooling2D(pool_size=(2, 2))(x)  # 池化层

# 添加更多的卷积层
x = SparseConv2D(filters=64, kernel_size=3, p=1, name='sparse_conv2d_2')(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = SparseConv2D(filters=128, kernel_size=3, p=1, name='sparse_conv2d_3')(x)
x = layers.Activation('relu')(x)
x = layers.Flatten()(x)  # 全局最大池化层

# 添加更多的全连接层
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)  # 输出层，假设输出为100个类别

model = models.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
sparse_conv2d_1 (SparseConv2 (None, 32, 32, 32)        898       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
sparse_conv2d_2 (SparseConv2 (None, 16, 16, 64)        18498     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0     

In [6]:
class UpdatePSparsity(tf.keras.callbacks.Callback):
    def __init__(self, model, sparsity_schedule):
        super(UpdatePSparsity, self).__init__()
        self.model = model
        self.sparsity_schedule = sparsity_schedule

    def on_epoch_end(self, epoch, logs=None):
        for layer_name, new_p in self.sparsity_schedule.items():
            layer = self.model.get_layer(name=layer_name)
            if epoch < len(new_p):
                p_value = new_p[epoch]
            else:
                p_value = new_p[-1]  # Use the last value for epochs beyond the predefined ones
            layer.update_p(p_value)
            #print(f"\nEpoch {epoch + 1}: Updated layer {layer_name} sparsity p to {p_value}")

sparsity_schedule = {
    'sparse_conv2d_1': [1],
    'sparse_conv2d_2': [1],
    'sparse_conv2d_3': [0.5]
}

In [7]:
# 编译模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# 训练模型，并在验证集上验证
model.fit(train_images, train_labels, epochs=40, batch_size=128, validation_data=(test_images, test_labels), callbacks=[UpdatePSparsity(model, sparsity_schedule)])

Epoch 1/40
391/391 [==============================] - 4s 4ms/step - loss: 3.7947 - accuracy: 0.1232 - val_loss: 3.2131 - val_accuracy: 0.2198
Epoch 2/40
391/391 [==============================] - 2s 5ms/step - loss: 3.3002 - accuracy: 0.2067 - val_loss: 3.0662 - val_accuracy: 0.2542
Epoch 3/40
391/391 [==============================] - 2s 5ms/step - loss: 2.9929 - accuracy: 0.2590 - val_loss: 2.8353 - val_accuracy: 0.2975
Epoch 4/40
391/391 [==============================] - 2s 5ms/step - loss: 2.7873 - accuracy: 0.3005 - val_loss: 2.7180 - val_accuracy: 0.3220
Epoch 5/40
391/391 [==============================] - 2s 5ms/step - loss: 2.6491 - accuracy: 0.3294 - val_loss: 2.6152 - val_accuracy: 0.3351
Epoch 6/40
391/391 [==============================] - 2s 5ms/step - loss: 2.5214 - accuracy: 0.3564 - val_loss: 2.5898 - val_accuracy: 0.3468
Epoch 7/40
391/391 [==============================] - 2s 5ms/step - loss: 2.4115 - accuracy: 0.3792 - val_loss: 2.4947 - val_accuracy: 0.3651
Epoch 

KeyboardInterrupt: 